In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)
!nvidia-smi

In [ ]:
img_size = (1200, 1)
batch_size = 64
direc = r'C:/Users/utkar/Downloads/flattened_data_grayscale/'
#test_dir = r'C:\Users/utkar/Desktop/ML/pytorch/captcha_web_extension/test_data'

ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    direc,
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training"
)

ds_val = tf.keras.preprocessing.image_dataset_from_directory(
    direc,
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="validation"
)

normalization_layer = layers.Rescaling(1./255)
n_train_ds = ds_train.map(lambda x, y: (tf.squeeze(normalization_layer(x)), y))
n_val_ds = ds_val.map(lambda x, y: (tf.squeeze(normalization_layer(x)), y))

train_img_batch, train_labels_batch = next(iter(n_train_ds))
val_img_batch, val_labels_batch = next(iter(n_val_ds))

In [ ]:
img = np.array(train_img_batch[0])
#img = img.reshape(40, 30)>0.5
print(img.reshape(40, 30))
plt.imshow(img.reshape(40, 30), cmap='gray')

In [ ]:
class_names = ds_train.class_names
print(class_names)

In [ ]:
labels_map = {
    0:'2', 1:'3', 2:'4', 3:'5', 4:'6', 5:'7', 6:'8', 7:'9', 8:'A', 9:'B', 
    10:'C', 11:'D', 12:'E', 13:'F', 14:'G', 15:'H', 16:'J', 17:'K', 18:'L', 
    19:'M', 20:'P', 21:'Q', 22:'R', 23:'S', 24:'T', 25:'U', 26:'V', 27:'W', 28:'X', 
    29:'Y', 30:'a', 31:'b', 32:'c', 33:'d', 34:'e', 35:'f', 36:'h', 37:'j', 
    38:'k', 39:'m', 40:'n', 41:'p', 42:'q', 43:'r', 44:'s', 45:'t', 46:'u', 47:'v', 
    48:'w', 49:'x', 50:'y'
}


In [ ]:
plt.figure(figsize=(10, 10))
a = 0
b1 = np.random.randint(0, 76)
for images, labels in n_train_ds.take(b1):
    for i in range(9):
        plt.subplot(3, 3, 1+i)
        img = np.array(images[i], dtype='float32')
        #print(labels_map[np.argmax(model(img.reshape(1, 1200)))])
        plt.imshow(img.reshape(40, 30), cmap='gray')
        #plt.title(class_names[labels[i]])
        plt.axis("off")
    break
            
plt.show()

In [ ]:
#print(labels_map[np.argmax(model(img.reshape(1, 1200)))])
img = img.reshape(40, 30)>0.5
plt.imshow(img.reshape(40, 30), cmap='gray')

In [ ]:
for img_batch, labels_batch in zip(train_img_batch, train_labels_batch):
    print(img_batch.shape)
    print(labels_batch.shape)
    break
print(train_img_batch.shape, train_labels_batch.shape)

In [ ]:
num_classes = 51

model = Sequential()
model.add(layers.Dense(128, input_dim=40*30, activation='relu'))
model.add(layers.Dense(51, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
epochs = 30
training = model.fit(
    n_train_ds,
    validation_data=n_val_ds,
    epochs=epochs
)

In [ ]:
model.save('final_model.h5')

In [ ]:
#model = keras.models.load_model("nn.h5")
acc = training.history['accuracy']
val_acc = training.history['val_accuracy']

loss = training.history['loss']
val_loss = training.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### I ran the below two commands on colab because the version of tf that I have does not allow me to use it

In [ ]:
import tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format keras '/final_model.h5' '/final_model'